In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import json, os, time
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

In [4]:
#Grab our API key from our json file, and set it to the tmdb api key

with open('/Users/jfabe/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
    
tmdb.API_KEY = login['api-key']

In [5]:
#Quick test to verify our API Key is working

movie = tmdb.Movies(603)
movie.info()

{'adult': False,
 'backdrop_path': '/l4QHerTSbMI7qgvasqxP36pqjN6.jpg',
 'belongs_to_collection': {'id': 2344,
  'name': 'The Matrix Collection',
  'poster_path': '/bV9qTVHTVf0gkW0j7p7M0ILD4pG.jpg',
  'backdrop_path': '/bRm2DEgUiYciDw3myHuYFInD7la.jpg'},
 'budget': 63000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'http://www.warnerbros.com/matrix',
 'id': 603,
 'imdb_id': 'tt0133093',
 'original_language': 'en',
 'original_title': 'The Matrix',
 'overview': 'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.',
 'popularity': 70.175,
 'poster_path': '/f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg',
 'production_companies': [{'id': 79,
   'logo_path': '/tpFpsqbleCzEE2p5EgvUq6ozfCA.png',
   'name': 'Village Roadshow Pictures',
   'origin_country': 'US'},
  {'id': 174,
   'logo_path': '/IuAlhI9eVC9Z8UQWOIDdWRKSEJ.png'

In [8]:
#The .info() does not contain the movie rating, which we will need
#Let's create a function that will add movie rating (certification) to our .info()
#Adapted from CD LP & https://github.com/celiao/tmdbsimple

def get_movie_with_rating(movie_id):
    movie = tmdb.Movies(movie_id)
    
    info = movie.info()
    
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1']=='US':
            info['certification'] = c['certification']
            
    return info        

In [17]:
#Let's test our function quick - first on "The Avengers"

get_movie_with_rating('tt0848228')

{'adult': False,
 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 205.168,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path

In [18]:
#One additional test - this time on "The Notebook"

get_movie_with_rating('tt0332280')

{'adult': False,
 'backdrop_path': '/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg',
 'belongs_to_collection': None,
 'budget': 29000000,
 'genres': [{'id': 10749, 'name': 'Romance'}, {'id': 18, 'name': 'Drama'}],
 'homepage': 'http://www.newline.com/properties/notebookthe.html',
 'id': 11036,
 'imdb_id': 'tt0332280',
 'original_language': 'en',
 'original_title': 'The Notebook',
 'overview': "An epic love story centered around an older man who reads aloud to a woman with Alzheimer's. From a faded notebook, the old man's words bring to life the story about a couple who is separated by World War II, and is then passionately reunited, seven years later, after they have taken different paths.",
 'popularity': 56.454,
 'poster_path': '/rNzQyW4f8B8cQeg7Dgj3n6eT5k9.jpg',
 'production_companies': [{'id': 12,
   'logo_path': '/iaYpEp3LQmb8AfAtmTvpqd4149c.png',
   'name': 'New Line Cinema',
   'origin_country': 'US'},
  {'id': 1565, 'logo_path': None, 'name': 'Avery Pix', 'origin_country': 'US'},
  {'id': 26

In [12]:
#Define our folder variable and verify our original data is there from step 1
folder = "Data/"
os.listdir(folder)

['.ipynb_checkpoints',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz']

In [13]:
#Create a new function for writing a json file
#Adapted from CD LP and https://www.geeksforgeeks.org/append-to-json-file-using-python/

def write_json(new_data, filename):
    with open(filename, 'r+') as f:
        file_data = json.load(f)
        
        if (type(new_data)==list) & (type(file_data)==list):
            file_data.extend(new_data)
            
        else:
            file_data.append(new_data)
            
            
        f.seek(0)
        json.dump(file_data, f)

In [15]:
#Reload our basics dataframe from step 1

basics = pd.read_csv('Data/title_basics.csv.gz')
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


In [16]:
#Create a list with years to extract from our API call
#Create an empty list to append errors to

years_to_get = [2000,2001]
errors  = []

In [20]:
#Now let's create an outer and inner for-loop to run through our API calls for our two years

for YEAR in tqdm_notebook(years_to_get, desc='Years', position=0):
    
    JSON_FILE = f'{folder}tmdb_api_results_{YEAR}.json'
    file_exists = os.path.isfile(JSON_FILE)
    
    if file_exists == False:
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}], f)
            
            
    df = basics.loc[basics['startYear']==YEAR].copy()
    movie_ids = df['tconst'].copy()
    
    previous_df = pd.read_json(JSON_FILE)
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
    
    for movie_id in tqdm_notebook(movie_ids_to_get, desc=f'Movies from {YEAR}', position=1, leave=True):
        try:
            temp = get_movie_with_rating(movie_id)
            write_json(temp, JSON_FILE)
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])
            
            
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f'{folder}final_tmdb_data_{YEAR}.csv.gz', compression="gzip", index=False)
    
print(f'- Total errors: {len(errors)}')        

Years:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1412 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1530 [00:00<?, ?it/s]

- Total errors: 446
